This script will iterate in all the folders in the same directory and look for pcaps. It will merge the contents of 5 folders at a time, generating a new pcapng inside the directory pcaps. Then it will remove any duplicate they have, and put the new files in the dir cleaned_pcaps. 
If you already don't have duplicated pcaps you can put them all in cleaned_pcaps and skip a few cells

`wireshark-cli` must be installed

In [ ]:
import os

from pathlib import Path
from concurrent.futures import ThreadPoolExecutor


NTHREADS = 8

Take all the separated pcaps, merge them into a couple of files.
This is an incredibly slow process; it took me 3+ hours.
Consider running it overnight.

In [ ]:
args = ""
i = 0
with ThreadPoolExecutor(NTHREADS) as executor:
    for dir in Path(".").iterdir():
        if dir.is_file():
            continue
        args += f"{dir.name}/* "
        i += 1
        if i % 5 == 0:
            command = f"mergecap -w out{i/5:.0f}.pcapng {args}"
            print(command)
            executor.submit(os.system, command)
            args = ""
command = f"mergecap -w pcaps/out{i/5:.0f}.pcapng {args}"
print(command)
os.system(command)

Remove most of the duplicate packets (there's plenty).
Take into account that the biggest files are more than 5G, so keep in mind that 4 threads at a time might be too much. I had a freeze with 4. If you have 16G of RAM in your system, I suggest tuning it down to max 2 threads. If you have 8, make it 1.

In [ ]:
# 1m 54.5s
with ThreadPoolExecutor(NTHREADS) as executor:
    for filename in Path("pcaps").iterdir():
        cmd = f"editcap -F pcap -d {filename} cleaned_pcaps/{filename.stem}_cleaned.pcap"
        executor.submit(os.system, cmd)


Merge all the cleaned files into a single pcapng. Why not do this first? There's so many duplicates that it would make for an enormous file (which also means high ram consumption that is not possible to parallelize)
I went from 131.8G to 22.2G.  

In [ ]:
# 1m 16.8s
cmd = f"mergecap -w out_final.pcap cleaned_pcaps/*.pcap"
print(f"Executing `{cmd}`")
os.system(cmd)
print(f"Done merging all pcaps.")

Clean the final pcap for good. I went from 22.2G to 9.9G

In [ ]:
# 1m 22.9s
filename = Path("out_final.pcap")
cmd = f"editcap -F pcap -d {filename} {filename.stem}_cleaned.pcap"
print(f"Executing `{cmd}`")
os.system(cmd)


Split to multiple pcaps of less than 1G for CICFlowMeter (It doesn't handle large files well https://github.com/ahlashkari/CICFlowMeter/issues/119/)

After this, use the fixed version of CICFlowMeter to compute the flow dataset from cleaned_pcaps/out_final_cleaned.pcapng
